<a href="https://colab.research.google.com/github/anjali-ojha/deep-learning/blob/a10/assignment/assignment-10/Anjali_Ojha_HW_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT for QnA:

# 1. Load the SQUAD 2.0 (https://pytorch.org/text/stable/datasets.html#torchtext.datasets.SQuAD2) dataset from pytorch dataset library. take minimum 20 QnA pairs.




In [1]:
! pip install datasets
! pip install portalocker
! pip install transformers datasets torch
! pip install transformers[torch]
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3

In [38]:
from datasets import load_dataset

def load_data():
  # Load the SQuAD 2.0 dataset
  squad_dataset = load_dataset("squad_v2")

  # Get the training data
  train_data = squad_dataset["train"]

  # Get the development data
  dev_data = squad_dataset["validation"]
  return train_data, dev_data


train_data, dev_data = load_data()
train_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [3]:
train_data.features

{'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [4]:
train_data['answers'][:2]

[{'text': ['in the late 1990s'], 'answer_start': [269]},
 {'text': ['singing and dancing'], 'answer_start': [207]}]

# 2. Display a few raw QnA data samples.


In [22]:
for i in range(5):
  print(f"\nsample = {i+1}, \n\t Ques = {train_data['question'][i]}, \n\t Ans  = {train_data['answers'][i]['text'][0]}")
# train_data['question'][:2]


sample = 1, 
	 Ques = When did Beyonce start becoming popular?, 
	 Ans  = in the late 1990s

sample = 2, 
	 Ques = What areas did Beyonce compete in when she was growing up?, 
	 Ans  = singing and dancing

sample = 3, 
	 Ques = When did Beyonce leave Destiny's Child and become a solo singer?, 
	 Ans  = 2003

sample = 4, 
	 Ques = In what city and state did Beyonce  grow up? , 
	 Ans  = Houston, Texas

sample = 5, 
	 Ques = In which decade did Beyonce become famous?, 
	 Ans  = late 1990s


# 3. Preprocess/Clean dataset as per BERT format.

In [44]:
from transformers import BertTokenizer, BertForQuestionAnswering

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

N = 1000

def preprocess_qa(example):
    question, context, answer = example['question'], example['context'], example['answers']['text'][0]
    inputs = tokenizer(question, context, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    inputs.update({'labels': tokenizer(answer, padding='max_length', truncation=True, max_length=32, return_tensors='pt')['input_ids']})
    return inputs


# Define your filter function
def filter_function(example):
    return len(example['answers']['text']) > 0


train_data, dev_data = load_data()

# Filter the train data
filtered_train_data = train_data.select(range(N)).filter(filter_function)
filtered_train_data
train_data = filtered_train_data.map(preprocess_qa)


# Filter the train data
filtered_dev_data = dev_data.select(range(N)).filter(filter_function)
dev_data = filtered_dev_data.map(preprocess_qa)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
train_data.select(range(100))

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100
})


# 4. Train the BERT QnA model. Evaluate the model.


In [47]:
# import torch
# from transformers import BertForQuestionAnswering, BertTokenizer
# from torch.utils.data import DataLoader
# from transformers import AdamW, get_linear_schedule_with_warmup
# from datasets import load_metric

# # Initialize BERT and tokenizer
# model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Define optimizer and scheduler
# optimizer = AdamW(model.parameters(), lr=5e-5)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_data) * 3)

# import torch

# # Check if CUDA is available (i.e., GPU is available)
# if torch.cuda.is_available():
#     device = torch.device("cuda")
#     print("CUDA is available. Using GPU.")
# else:
#     device = torch.device("cpu")
#     print("CUDA is not available. Using CPU.")

# print("Device:", device)

# def train_epoch(model, dataloader, optimizer, scheduler, device):
#     model.train()
#     total_loss = 0
#     for batch in dataloader:
#         optimizer.zero_grad()
#         print(batch['input_ids'][0][0])
#         input_ids = batch['input_ids'][0][0].to(device)  # Assuming 'input_ids' is a list containing tensors
#         attention_mask = batch['attention_mask'][0].to(device)  # Assuming 'attention_mask' is a list containing tensors
#         start_positions = batch['start_positions'][0].to(device)  # Assuming 'start_positions' is a list containing tensors
#         end_positions = batch['end_positions'][0].to(device)  # Assuming 'end_positions' is a list containing tensors
#         outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
#         loss = outputs.loss
#         total_loss += loss.item()
#         loss.backward()
#         optimizer.step()
#         scheduler.step()
#     return total_loss / len(dataloader)


# def evaluate_model(model, dataloader, device):
#     model.eval()
#     metric = load_metric("squad_v2")
#     for batch in dataloader:
#         with torch.no_grad():
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             start_positions = batch['start_positions'].to(device)
#             end_positions = batch['end_positions'].to(device)
#             outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
#         predictions = tokenizer.batch_decode(torch.argmax(outputs.start_logits, axis=1))
#         metric.add_batch(predictions=predictions, references=batch['answers'])
#     return metric.compute()


# # Set up data loaders
# train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
# dev_dataloader = DataLoader(dev_data, batch_size=8, shuffle=False)

# # Training loop
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# for epoch in range(3):  # Number of epochs
#     train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)
#     print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f}")

# # Evaluate the model on the dev set
# evaluation_result = evaluate_model(model, dev_dataloader, device)
# print("Evaluation Result:", evaluation_result)

import torch
from transformers import BertForQuestionAnswering, BertTokenizer
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from datasets import load_metric

# Initialize BERT and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_data) * 3)

# Assuming 'train_data' and 'dev_data' are your training and validation datasets respectively
# train_data = []  # Placeholder for your training data
# dev_data = []  # Placeholder for your validation data

# Set up data loaders
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
dev_dataloader = DataLoader(dev_data, batch_size=8, shuffle=False)

# Check if CUDA is available (i.e., GPU is available)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

print("Device:", device)

def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'][0][0].to(device)
        attention_mask = batch['attention_mask'][0][0].to(device)
        start_positions = batch['answers']['answer_start'][0][0].to(device)
        end_positions = batch['answers']['answer_start'] + torch.tensor([len(answer['text']) for answer in batch['answers']]).to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
    return total_loss / len(dataloader)

def evaluate_model(model, dataloader, device):
    model.eval()
    metric = load_metric("squad_v2")
    for batch in dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['answers']['answer_start'].to(device)
            end_positions = batch['answers']['answer_start'] + torch.tensor([len(answer['text']) for answer in batch['answers']]).to(device)
            outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        predictions = tokenizer.batch_decode(torch.argmax(outputs.start_logits, axis=1))
        metric.add_batch(predictions=predictions, references=batch['answers']['text'])
    return metric.compute()

# Training loop
model.to(device)

for epoch in range(3):  # Number of epochs
    train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)
    print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f}")

# Evaluate the model on the dev set
evaluation_result = evaluate_model(model, dev_dataloader, device)
print("Evaluation Result:", evaluation_result)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


CUDA is available. Using GPU.
Device: cuda


TypeError: string indices must be integers

# 5. Perform an Inference and show the predicted vs ground truth answers.

In [ ]:
results = trainer.evaluate()
print(results)

In [ ]:
! pip install transformers[torch]

In [35]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Load SQUAD 2.0 dataset
dataset = load_dataset("squad_v2")
qa_pairs = dataset["train"][:20]  # Take a subset of at least 20 Q&A pairs
print(qa_pairs)
# Display raw Q&A data samples
for example in qa_pairs:
    print("Question:", example["question"])
    print("Answer:", example["answers"]["text"])
    print()

# Preprocess/Clean dataset as per BERT format
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 512  # Maximum length of input tokens for BERT

def preprocess_qa_pairs(qa_pairs):
    inputs = tokenizer(qa_pairs["question"], qa_pairs["context"], truncation=True, max_length=max_length, padding="max_length")
    labels = {
        "start_positions": [],
        "end_positions": []
    }
    for answer in qa_pairs["answers"]:
        start_char = answer["answer_start"]
        end_char = start_char + len(answer["text"])
        start_token = inputs.char_to_token(qa_pairs["input_ids"][0], start_char)
        end_token = inputs.char_to_token(qa_pairs["input_ids"][0], end_char - 1)
        labels["start_positions"].append(start_token)
        labels["end_positions"].append(end_token)
    return inputs, labels

processed_data = [preprocess_qa_pairs(example) for example in qa_pairs]

# Convert processed data to DataLoader
inputs = torch.tensor([example[0]["input_ids"] for example in processed_data])
attention_masks = torch.tensor([example[0]["attention_mask"] for example in processed_data])
start_positions = torch.tensor([example[1]["start_positions"] for example in processed_data])
end_positions = torch.tensor([example[1]["end_positions"] for example in processed_data])

train_inputs, val_inputs, train_masks, val_masks, train_starts, val_starts, train_ends, val_ends = train_test_split(
    inputs, attention_masks, start_positions, end_positions, random_state=42, test_size=0.2
)

train_dataset = TensorDataset(train_inputs, train_masks, train_starts, train_ends)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Initialize and train the BERT QnA model
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(3):
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        optimizer.zero_grad()
        input_ids, attention_mask, start_positions, end_positions = batch
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
val_dataset = TensorDataset(val_inputs, val_masks, val_starts, val_ends)
val_loader = DataLoader(val_dataset, batch_size=8)

for batch in val_loader:
    input_ids, attention_mask, start_positions, end_positions = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    predicted_starts = torch.argmax(outputs.start_logits, dim=1)
    predicted_ends = torch.argmax(outputs.end_logits, dim=1)
    # Compare predicted_starts and predicted_ends with ground truth for evaluation metrics

# Inference
example = qa_pairs[0]  # Take an example for inference
input_text = example["question"] + " " + example["context"]
inputs = tokenizer(input_text, truncation=True, max_length=max_length, padding="max_length", return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
start_index = torch.argmax(outputs.start_logits)
end_index = torch.argmax(outputs.end_logits)
predicted_answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index+1])

print("Question:", example["question"])
print("Predicted Answer:", predicted_answer)
print("Ground Truth Answer:", example["answers"]["text"])


{'id': ['56be85543aeaaa14008c9063', '56be85543aeaaa14008c9065', '56be85543aeaaa14008c9066', '56bf6b0f3aeaaa14008c9601', '56bf6b0f3aeaaa14008c9602', '56bf6b0f3aeaaa14008c9603', '56bf6b0f3aeaaa14008c9604', '56bf6b0f3aeaaa14008c9605', '56d43c5f2ccc5a1400d830a9', '56d43c5f2ccc5a1400d830aa', '56d43c5f2ccc5a1400d830ab', '56d43c5f2ccc5a1400d830ac', '56d43c5f2ccc5a1400d830ad', '56d43ce42ccc5a1400d830b4', '56d43ce42ccc5a1400d830b5', '56be86cf3aeaaa14008c9076', '56be86cf3aeaaa14008c9078', '56be86cf3aeaaa14008c9079', '56bf6e823aeaaa14008c9627', '56bf6e823aeaaa14008c9629'], 'title': ['Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé', 'Beyoncé'], 'context': ['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she p

TypeError: string indices must be integers